In [1]:
import pandas as pd
import os
import csv

import math
import numpy as np
import matplotlib.pyplot as plt

读取训练集和测试集的csv文件

In [2]:
train_data = pd.read_csv('./airfoil_noise_samples.csv')
test_data = pd.read_csv('./airfoil_noise_test.csv')
train_data.head()

,Frequency,Angle,Displacement,Chord length,Velocity,Thickness,Sound Pressure
0,2175.611424,15.138865,21.075119,0.088194,66.764401,0.044771,122.365215
1,2962.923620,13.400893,13.200889,0.048462,78.221903,0.011041,129.296236
2,4430.810843,2.164599,13.959536,0.226743,57.053201,0.011499,121.827380
3,4939.695645,13.857682,18.203793,0.021705,23.896377,0.021475,114.998132
4,2193.979785,9.298757,11.007713,0.052737,38.917034,0.001741,125.639641


定义特征和目标变量

In [3]:
X_train = train_data.iloc[:, :-1]
Y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
Y_test = test_data.iloc[:, -1]

数据归一化，防止数据过拟合

In [4]:
#归一化
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()
X_train.head()

,Frequency,Angle,Displacement,Chord length,Velocity,Thickness
0,-0.275324,1.240614,1.602819,-0.558959,0.934190,2.181587
1,0.007620,0.928480,-0.074895,-0.989410,1.631190,-0.232215
2,0.535150,-1.089521,0.086745,0.942044,0.343425,-0.199430
3,0.718034,1.010518,0.991043,-1.279290,-1.673619,0.514487
4,-0.268723,0.191750,-0.542182,-0.943098,-0.759861,-0.897771


使用Elastic Nets训练

In [5]:
#设置学习率和迭代次数
learning_rate = 0.001
epochs = 4000
alphas = [0.1, 0.5, 0.9]
lmbdas = [0.001, 0.01, 0.1, 1, 10]

best_alpha = None
best_lmbda = None
best_mse = float('inf')

#Lasso惩罚加ridge惩罚
def compute_cost(X, y, w, b, alpha, lmbda):
    m = len(y)
    predictions = X.dot(w) + b
    cost = (1/2*m) * np.sum(np.square(predictions-y))
    lasso_penalty = lmbda * np.sum(np.abs(w))
    ridge_penalty = lmbda * np.sum(np.square(w))
    return cost + alpha * lasso_penalty + (1 - alpha) * ridge_penalty

# Gradient descent
def gradient_descent(X, y, w, b, alpha, lmbda, learning_rate, epochs):
    m = len(y)
    for _ in range(epochs):
        predictions = X.dot(w) + b
        dw = (1/m) * X.T.dot(predictions - y) + lmbda * (alpha * np.sign(w) + (1 - alpha) * w)
        db = (1/m) * np.sum(predictions - y)
        w -= learning_rate * dw
        b -= learning_rate * db
    return w, b

# 5折交叉验证调整alpha
def cross_validate(X, y, alpha, lmbda):
    fold_size = len(y) // 5
    mse_list = []
    
    for i in range(5):
        start = i * fold_size
        end = start + fold_size

        X_val = X[start:end]
        y_val = y[start:end]
        X_train_fold = np.concatenate((X[:start], X[end:]))
        y_train_fold = np.concatenate((y[:start], y[end:]))

        w = np.zeros(X_train_fold.shape[1])
        b = 0
        w, b = gradient_descent(X_train_fold, y_train_fold, w, b, alpha, lmbda, learning_rate, epochs)
        predictions = X_val.dot(w) + b
        mse = np.mean((predictions - y_val)**2)
        mse_list.append(mse)
        
    return np.mean(mse_list)


for alpha in alphas:
    for lmbda in lmbdas:
        mse = cross_validate(X_train, Y_train, alpha, lmbda)
        if mse < best_mse:
            best_mese = mse
            best_alpha = alpha
            best_lmbda = lmbda

print(f"最佳的alpha是: {best_alpha}")
print(f"最佳的lambda是: {best_lmbda}")

最佳的alpha是: 0.9
最佳的lambda是: 10


In [6]:
#初始化w，b
w = np.zeros(X_train.shape[1])
b = 0
w, b = gradient_descent(X_train, Y_train, w, b, best_alpha, best_lmbda, learning_rate, epochs)

使用R^2分数和MSE来评估模型在测试集上的表现：

In [7]:
#计算mse和r2评分
predictions = X_test.dot(w) + b
mse_test = np.mean((predictions - Y_test)**2)
r2 = 1 - mse_test / np.var(Y_test)

# 输出结果
print(f"MSE on test set: {mse_test}")
print(f"R^2 Score on test set: {r2}")

MSE on test set: 26.720710800551238
R^2 Score on test set: -0.23455010266006315
